# Heading:
Here all the initialization is done, like downloading the data

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1TvK7RQvwrMTJQecOvDzqjJ3GV2cY7R_I'})
download.GetContentFile('/content/testData.zip')

In [69]:
#unzip the downloaded content
!rm -r content/
!unzip /content/testData.zip

rm: cannot remove 'content/': No such file or directory
Archive:  /content/testData.zip
   creating: content/JAAD/test/
   creating: content/JAAD/test/vbb/
  inflating: content/JAAD/test/vbb/video_0128_296.xml  
  inflating: content/JAAD/test/vbb/video_0113_188.xml  
  inflating: content/JAAD/test/vbb/video_0113_75.xml  
  inflating: content/JAAD/test/vbb/video_0324_39.xml  
  inflating: content/JAAD/test/vbb/video_0124_344.xml  
  inflating: content/JAAD/test/vbb/video_0327_141.xml  
  inflating: content/JAAD/test/vbb/video_0113_116.xml  
  inflating: content/JAAD/test/vbb/video_0113_171.xml  
  inflating: content/JAAD/test/vbb/video_0210_108.xml  
  inflating: content/JAAD/test/vbb/video_0124_208.xml  
  inflating: content/JAAD/test/vbb/video_0128_150.xml  
  inflating: content/JAAD/test/vbb/video_0111_119.xml  
  inflating: content/JAAD/test/vbb/video_0122_86.xml  
  inflating: content/JAAD/test/vbb/video_0128_257.xml  
  inflating: content/JAAD/test/vbb/video_0322_50.xml  
  inflat

In [70]:
!pip install filterpy
!pip install scikit-learn
!pip install scikit-image
!pip install opencv-python

# Imports
Here all the imports are done

In [0]:
from __future__ import print_function

from numba import jit
import os.path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io
from sklearn.utils.linear_assignment_ import linear_assignment
import glob
import time
import argparse
from filterpy.kalman import KalmanFilter
import cv2
import pandas as pd
import csv
import os

# Data Conversion of YOLO output to SORT input

In [91]:
!mkdir output
!rm -r input/vid
!mkdir input/vid/
#choose the video file accordingly and select all of its frames with wildcards. Here the video 111 is choosen
#here it is imortant that you choose the same video two cells further down (maybe do search and replace)
!cp content/JAAD/test/vid/video_0111_*.jpg input/vid

mkdir: cannot create directory ‘output’: File exists
rm: cannot remove 'input/vid': No such file or directory


In [0]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]

#alist.sort(key=natural_keys)

In [0]:
import os
import glob

if not os.path.exists('input'):
   os.makedirs('input')

path = "content/JAAD/test/predict_yoloJAAD"
#listing = os.listpath(folder)
listing = glob.glob(os.path.join(path, 'video_0111_*.txt'))
listing.sort(key=natural_keys) 

columns = ["frame", "detect", "x", "y", "width", "height", "confidence", "ignore1", "ignore2", "ignore3"]
df = pd.DataFrame(columns=columns)
for n,file in enumerate(listing):
    temp_df = pd.read_table(file, delim_whitespace=True, header = None, names=('class', 'confidence', 'x_or','y_or','x_ul','y_ul'))
    rows = temp_df.shape[0]
    new_df = pd.DataFrame(-1, index=np.arange(rows), columns=columns)
    new_df["frame"] = n+1
    new_df["x"] = temp_df["x_or"]
    new_df["y"] = temp_df["y_or"]
    new_df["width"] = temp_df["x_ul"]-temp_df["x_or"]
    new_df["height"] = temp_df["y_ul"]-temp_df["y_or"]
    new_df["confidence"] = temp_df["confidence"]
    df = pd.concat([df,new_df])
df.to_csv('input/result.txt',index=False,header=False)

# SORT Algorithm
Implementation of the SORT Algorithm

In [0]:


@jit
def iou(bb_test,bb_gt):
  """
  Computes IUO between two bboxes in the form [x1,y1,x2,y2]
  """
  xx1 = np.maximum(bb_test[0], bb_gt[0])
  yy1 = np.maximum(bb_test[1], bb_gt[1])
  xx2 = np.minimum(bb_test[2], bb_gt[2])
  yy2 = np.minimum(bb_test[3], bb_gt[3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[2]-bb_test[0])*(bb_test[3]-bb_test[1])
    + (bb_gt[2]-bb_gt[0])*(bb_gt[3]-bb_gt[1]) - wh)
  return(o)

def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2]-bbox[0]
  h = bbox[3]-bbox[1]
  x = bbox[0]+w/2.
  y = bbox[1]+h/2.
  s = w*h    #scale is just area
  r = w/float(h)
  return np.array([x,y,s,r]).reshape((4,1))

def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2]*x[3])
  h = x[2]/w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))


class KalmanBoxTracker(object):
  """
  This class represents the internel state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4)
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)

def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)

  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)
  iou_matrix = np.zeros((len(detections),len(trackers)),dtype=np.float32)

  for d,det in enumerate(detections):
    for t,trk in enumerate(trackers):
      iou_matrix[d,t] = iou(det,trk)
  matched_indices = linear_assignment(-iou_matrix)

  unmatched_detections = []
  for d,det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t,trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0],m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)



class Sort(object):
  def __init__(self,max_age=1,min_hits=3):
    """
    Sets key parameters for SORT
    """
    self.max_age = max_age
    self.min_hits = min_hits
    self.trackers = []
    self.frame_count = 0

  def update(self,dets):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections.
    Returns the a similar array, where the last column is the object ID.

    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    #get predicted locations from existing trackers.
    trks = np.zeros((len(self.trackers),5))
    to_del = []
    ret = []
    for t,trk in enumerate(trks):
      pos = self.trackers[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
      if(np.any(np.isnan(pos))):
        to_del.append(t)
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks)

    #update matched trackers with assigned detections
    for t,trk in enumerate(self.trackers):
      if(t not in unmatched_trks):
        d = matched[np.where(matched[:,1]==t)[0],0]
        trk.update(dets[d,:][0])

    #create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        trk = KalmanBoxTracker(dets[i,:]) 
        self.trackers.append(trk)
    i = len(self.trackers)
    for trk in reversed(self.trackers):
        d = trk.get_state()[0]
        if((trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits)):
          ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
        i -= 1
        #remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers.pop(i)
    if(len(ret)>0):
      return np.concatenate(ret)
    return np.empty((0,5))
    
def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='SORT demo')
    parser.add_argument('--display', dest='display', help='Display online tracker output (slow) [False]',action='store_true')
    args = parser.parse_args()
    return args



In [0]:
def runSORT():
  # all train
  display = False
  phase = 'train'
  total_time = 0.0
  total_frames = 0
  colours = np.random.rand(32,3) #used only for display
    
  if not os.path.exists('output'):
    os.makedirs('output')
  

  mot_tracker = Sort() #create instance of the SORT tracker
  seq_dets = np.loadtxt('input/result.txt',delimiter=',') #load detections
  with open('output/result.txt','w') as out_file:

      for frame in range(int(seq_dets[:,0].max())):
        frame += 1 #detection and frame numbers begin at 1
        dets = seq_dets[seq_dets[:,0]==frame,2:7]
        dets[:,2:4] += dets[:,0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2]
        total_frames += 1

        start_time = time.time()
        trackers = mot_tracker.update(dets)
        cycle_time = time.time() - start_time
        total_time += cycle_time

        for d in trackers:
          print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]),file=out_file)
          
        
  print("Total Tracking took: %.3f for %d frames or %.1f FPS"%(total_time,total_frames,total_frames/total_time))
  

In [79]:
runSORT()

Total Tracking took: 1.045 for 270 frames or 258.3 FPS


# Post processing
Writing the bounding boxes to the frames and creating a videofile from the frames

In [0]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]

#alist.sort(key=natural_keys)
#print(alist)

In [0]:
def create_image_paths(directory): #input in this format: 'dir/subdir/picdir'
  imageFiles = os.listdir(directory)
  imageFiles.sort(key=natural_keys)
  #imageFiles = sorted(imageFiles, key=lambda x: float(os.path.split(x)[1][:-3]))
  imagePaths = imageFiles
  for n,img in enumerate(imageFiles):
    imagePaths[n] =  str(directory + '/' + imageFiles[n])  #possible place of error, if the folder is given the / already like this; picture/
    
  imageFiles = os.listdir(directory)
  imageFiles.sort(key=natural_keys)
  return (imageFiles, imagePaths)

In [0]:
def import_boundingboxes(boxPath): #returns a dictionary with an array containing the bounding boxes of every frame
  #with open(boxPath) as csvDataFile:
  csvReader = np.loadtxt(link,delimiter=',') #load detections
   #     csvReader = csv.reader(csvDataFile)
  boxData = {}
  n=1 #frame counter
  X = []
  for row in csvReader:
    if int(row[0])==int(n):
      X.append(row)
    else:          
      boxData[str(n)] = np.array(X)
      X = []
      X.append(row)
      n+=1
  return boxData           
            

In [0]:
##delete
def create_bounding_boxes(pictureDir, boxPath):
  
  # define text for labels
  font = cv2.FONT_HERSHEY_COMPLEX_SMALL
  fontScale = 1
  fontColor = (0,0,0)
  thickness = 1
  offsetY = -5
  
  boxData = import_boundingboxes(boxPath) #load the data of the bounding boxes  'output/ETH-Pedcross2.txt'
  (imageFiles,imagePaths) = create_image_paths(pictureDir) #load the paths to the image files

  #iterate over all images and add the bounding boxes
  for i in range(1,len(boxData)):
      #load a frame
      boxFrame = boxData[str(i)]
      image = cv2.imread(imagePaths[i-1])
     
      #add all the bounding boxes of the image
      for row in boxFrame:
        
        tl = (int(row[2]),int(row[3]))
        br = (int(row[2])+int(row[4]),int(row[3])+int(row[5]))
        font = cv2.FONT_HERSHEY_SIMPLEX
        image = cv2.rectangle(image,tl,br,(0,255,0))
        
        label = str(int(row[1]))
        confidence = str(row[6])
        text = '{0} | {1:.3f}'.format(label,confidence)
      
        (text_width, text_height) = cv2.getTextSize(text, font, fontScale=fontScale, thickness=thickness)[0]

          #display label with colored background
        txtBckgrnd = ((tl[0] - 2*thickness, tl[1] + offsetY + thickness), (tl[0] + offsetY + text_width - 2, tl[1] - text_height - 2))
        
        image = cv2.putText(image, text, tl, font, fontScale=fontScale, color=fontColor, thickness=thickness)
        #image = cv2.putText(image,string,tl, font, 1,(255,255,255),2,cv2.LINE_AA)
        
        #cv2.putText(image, label, (tl[0]-2,tl[0]-2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
    
      #write the image to file
      x = cv2.imwrite(str('output/vid/'+imageFiles[i-1]), image)
  
  return

In [0]:
def draw_bounding_box(image, text, br, tl):

    thickness = 2
    color = (255,255,255)
    cv2.rectangle(image, br, tl, color, thickness)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scaling = 1
    cv2.putText(image,text,br, font, font_scaling,color,4,cv2.LINE_AA)
   
   
    return image

In [0]:
##delete
image = cv2.imread('input/vid/video_0111_1.jpg')


br = (839,682)
tl = (839+28,682+45)

text = str('ID')

image = draw_bounding_box(image,text, br, tl)



x = cv2.imwrite('image.jpg', image)

In [0]:
def create_bounding_boxes(pictureDir, boxPath):
  
  if not os.path.exists('output/vid'):
    os.makedirs('output/vid')
  
  boxData = import_boundingboxes(boxPath) #load the data of the bounding boxes  'output/ETH-Pedcross2.txt'
  (imageFiles,imagePaths) = create_image_paths(pictureDir) #load the paths to the image files

  #iterate over all images and add the bounding boxes
  for i in range(1,len(boxData)):
      #load a frame
      boxFrame = boxData[str(i)]
      image = cv2.imread(imagePaths[i-1])
     
      #add all the bounding boxes of the image
      for row in boxFrame:
        br = (int(row[2]),int(row[3]))
        tl = (int(row[2])+int(row[4]),int(row[3])+int(row[5]))
        text = str(int(row[1]))
        
        #print(br)
        #print(tl)
        #print(text)
        
        image = draw_bounding_box(image,text, br, tl)
        x = cv2.imwrite('image.jpg', image)
      x = cv2.imwrite(str('output/vid/'+imageFiles[i-1]), image)
  return

In [0]:
def convert_frames_to_video(pathIn,pathOut,fps): #path in in format 'folder/subfo/imagefolder/'
    #load image files
    imageFiles = os.listdir(directory)
    imageFiles.sort(key=natural_keys)
    frame_array = []
    #files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #(imageFiles, imagePaths) = create_image_paths(directory)
    #for sorting the file names properly
    #files.sort(key = lambda x: int(x[5:-4]))
    
    emergency_frame_path = pathIn + imageFiles[0]
    for i,file in enumerate(imageFiles):
        filename=pathIn + file
        print(filename)
        #reading each files
        
        try:
          img = cv2.imread(filename)
          height, width, layers = img.shape
          size = (width,height)
        except: 
          img = cv2.imread(emergency_frame_path)
          height, width, layers = img.shape
          size = (width,height)
        
        
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
    

Here the bounding boxes from the prediction are drawn onto the images

In [92]:
directory = 'input/vid'
link = 'output/result.txt'
create_bounding_boxes(directory, link)


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


Here the video is created

In [93]:

from os.path import isfile, join
#pathIn= 'mot_benchmark/train/ETH-Pedcross2/img1/'
pathIn = 'output/vid/'
pathOut = 'video.avi'
fps = 25.0
convert_frames_to_video(pathIn, pathOut, fps)

output/vid/video_0111_1.jpg
output/vid/video_0111_2.jpg
output/vid/video_0111_3.jpg
output/vid/video_0111_4.jpg
output/vid/video_0111_5.jpg
output/vid/video_0111_6.jpg
output/vid/video_0111_7.jpg
output/vid/video_0111_8.jpg
output/vid/video_0111_9.jpg
output/vid/video_0111_10.jpg
output/vid/video_0111_11.jpg
output/vid/video_0111_12.jpg
output/vid/video_0111_13.jpg
output/vid/video_0111_14.jpg
output/vid/video_0111_15.jpg
output/vid/video_0111_16.jpg
output/vid/video_0111_17.jpg
output/vid/video_0111_18.jpg
output/vid/video_0111_19.jpg
output/vid/video_0111_20.jpg
output/vid/video_0111_21.jpg
output/vid/video_0111_22.jpg
output/vid/video_0111_23.jpg
output/vid/video_0111_24.jpg
output/vid/video_0111_25.jpg
output/vid/video_0111_26.jpg
output/vid/video_0111_27.jpg
output/vid/video_0111_28.jpg
output/vid/video_0111_29.jpg
output/vid/video_0111_30.jpg
output/vid/video_0111_31.jpg
output/vid/video_0111_32.jpg
output/vid/video_0111_33.jpg
output/vid/video_0111_34.jpg
output/vid/video_0111_3